In [1]:
from keras.models import Sequential, model_from_json
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.constraints import maxnorm
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, SeparableConv2D
from keras.layers import Bidirectional, RepeatVector, TimeDistributed, Permute
from keras.layers import concatenate, add, Lambda, multiply
from keras.layers import LSTM
from keras.models import Model
from keras.regularizers import l2

from keras.applications import imagenet_utils, Xception
from keras.preprocessing.image import img_to_array
import keras.backend as K

import numpy as np 
import json
import os
import cv2
import h5py
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [3]:
from helpers import TrainingMonitor
from helpers import Utils

In [4]:
Xoh_file = open('../input/dataset/Xoh.pickle', 'rb')
x = pickle.load(Xoh_file)
Xoh_file.close()

In [5]:
Yoh_file = open('../input/dataset/Yoh.pickle', 'rb')
y = pickle.load(Yoh_file)
Yoh_file.close()

In [6]:
x.shape

(1000, 30, 37)

In [7]:
y.shape

(1000, 10, 11)

In [8]:
train_split, val_split = 0.7, 0.15

train_len = int(train_split * x.shape[0])
train_val_len = int((train_split + val_split) * x.shape[0])

x_train = x[:train_len]
x_val = x[train_len:train_val_len]
x_test = x[train_val_len:]

y_train = y[:train_len]
y_val = y[train_len:train_val_len]
y_test = y[train_val_len:]

In [9]:
x_train.shape

(700, 30, 37)

In [10]:
y_train.shape

(700, 10, 11)

In [11]:
TIME_STEPS = x_train.shape[1]
INPUT_DIM = x_train.shape[2]

In [12]:
def attention(inputs):
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Dense(TIME_STEPS, activation='softmax')(a)
    a_probs = Permute((2, 1))(a)
    output_attention_mul = multiply([inputs, a_probs])
    return output_attention_mul

In [13]:
inputs = Input(shape=(30, 37,))
bi_lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True))(inputs)
attention_block = attention(bi_lstm)
output = Flatten()(attention_block)
output = RepeatVector(10)(output)
output = LSTM(128, dropout=0.5, return_sequences=True)(output)
output = TimeDistributed(Dense(11, activation="softmax"))(output)
model = Model(inputs, output)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30, 37)       0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 30, 128)      52224       input_1[0][0]                    
__________________________________________________________________________________________________
permute_1 (Permute)             (None, 128, 30)      0           bidirectional_1[0][0]            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128, 30)      930         permute_1[0][0]                  
__________________________________________________________________________________________________
permute_2 

In [14]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]) 

In [15]:
output_path = "../output/"

print(os.getpid())
filepath=output_path + "progress/seq2seq-stacked-weights-best.hdf5"
MC = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')

figPath = os.path.sep.join([output_path, "monitor/{}.png".format(os.getpid())])
jsonPath = os.path.sep.join([output_path, "monitor/{}.json".format(os.getpid())])
TM = TrainingMonitor(figPath, jsonPath=jsonPath, startAt=0)

RLR = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

callbacks = [MC, TM, RLR]

9452


In [16]:
history = model.fit(x_train, y_train,
            batch_size=32,
            epochs=100,
            validation_data=(x_val, y_val),
            shuffle="batch",
            callbacks=callbacks)

Train on 700 samples, validate on 150 samples
Epoch 1/100
700/700 [==============================] - 6s 8ms/step - loss: 2.2774 - acc: 0.1986 - val_loss: 2.1595 - val_acc: 0.2480
Epoch 2/100
700/700 [==============================] - 3s 4ms/step - loss: 2.1202 - acc: 0.2413 - val_loss: 2.0593 - val_acc: 0.1660
Epoch 3/100
700/700 [==============================] - 3s 4ms/step - loss: 2.0099 - acc: 0.2767 - val_loss: 1.9523 - val_acc: 0.3487
Epoch 4/100
700/700 [==============================] - 3s 4ms/step - loss: 1.8989 - acc: 0.3707 - val_loss: 1.8205 - val_acc: 0.4047
Epoch 5/100
700/700 [==============================] - 3s 4ms/step - loss: 1.7499 - acc: 0.4011 - val_loss: 1.6276 - val_acc: 0.4807
Epoch 6/100
700/700 [==============================] - 3s 4ms/step - loss: 1.5039 - acc: 0.4813 - val_loss: 1.3210 - val_acc: 0.4873
Epoch 7/100
700/700 [==============================] - 3s 4ms/step - loss: 1.2620 - acc: 0.4916 - val_loss: 1.2082 - val_acc: 0.4893
Epoch 8/100
700/700 [==

In [17]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Train: %.2f%%; Val: %.2f%%; Test: %.2f%%" % 
      (np.max(history.history['acc'])*100, np.max(history.history['val_acc'])*100, scores[1]*100)
     )

Train: 82.84%; Val: 82.47%; Test: 83.13%


In [ ]:
model_json = model.to_json()
with open("../output/saved/date_model_seq2seq_94.33.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("../output/saved/date_weight_seq2seq_94.33.hdf5")